In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2


def generate_refrence_frame(all_frames, alpha):
   
    reference_frame = all_frames[0]
    
    for frame in all_frames:
        
        # Compute the weighted sum of the current frame and the reference frame
        reference_frame = (alpha * frame + (1 - alpha) * reference_frame)
        
        frame_differences = np.abs(frame.astype(np.float32) - reference_frame.astype(np.float32)).astype(np.uint8)
        
    
#     plt.imshow(reference_frame, cmap = 'gray')
    
    return reference_frame.astype(np.uint8)
    
def dfs(img, visited, x, y, label):
    if x < 0 or x >= img.shape[0] or y < 0 or y >= img.shape[1] or visited[x, y] or img[x, y] == 0:
        return

    visited[x, y] = True
    img[x, y] = label

    # 8-connectivity neighbors
    neighbors = [(-1, -1), (-1, 0), (-1, 1),
                 (0, -1),           (0, 1),
                 (1, -1), (1, 0), (1, 1)]

    for dx, dy in neighbors:
        dfs(img, visited, x + dx, y + dy, label)

def find_contours(binary_image):
    labeled_image = binary_image.copy()
    visited = np.zeros_like(binary_image, dtype=bool)
    label_count = 1

    for x in range(binary_image.shape[0]):
        for y in range(binary_image.shape[1]):
            if not visited[x, y] and labeled_image[x, y] == 255:
                dfs(labeled_image, visited, x, y, label_count)
                label_count += 1

    return labeled_image

def count_objects(labeled_image):
    unique_labels = np.unique(labeled_image)
    object_count = len(unique_labels) - 1  # subtract background label
    return object_count

def count_moving_objects(video_path, reference_frame, threshold=30):
    cap = cv2.VideoCapture(video_path)

    object_counts = []

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate absolute difference between the current frame and the reference frame
        frame_diff = cv2.absdiff(frame_gray, reference_frame)

        # Apply threshold to classify pixels as moving or static
        _, thresholded_frame = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)

        # Find contours manually without using findContours
        labeled_frame = find_contours(thresholded_frame)

        # Count the number of moving objects
        object_count = count_objects(labeled_frame)
        object_counts.append(object_count)

        frame_count += 1

    cap.release()

    return object_counts




In [3]:
# Open the video file
video_capture = cv2.VideoCapture('Dataset//DatasetC.avi')  # Replace with your video file path

# Check if the video file is opened successfully
if not video_capture.isOpened():
    print("Error: Could not open video file.")
    exit()

# Loop through the frames and process each one

frames = []
while True:
    ret, frame = video_capture.read()

    if not ret:
        break  # Break the loop if we've reached the end of the video

    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frames.append(gray_image)


# # Release the video file and close the display window
video_capture.release()
cv2.destroyAllWindows()

# c) Generate a reference frame using weighted temporal averaging
reference_frame = generate_refrence_frame(frames, 0.027)

In [ ]:
# Example usage:




# d) Count the number of moving objects in each frame and plot the results
object_counts = count_moving_objects('Dataset//DatasetC.avi', reference_frame, threshold=30)
plot_object_counts(object_counts)